# Gathering Data

Importing requried librarys

In [1]:
%%writefile ../src/datautil.py
#datautil.py
import os
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

Overwriting ../src/datautil.py


Functions for Downloading data

In [2]:
%%writefile -a ../src/datautil.py

def get_soup(url):
    
    
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    return soup

def download_data(url, n=50, sleep=1):
    cpus = [dict() for x in range(n)]
    soup = get_soup(url)
    table = soup.find('div', class_='chart')
    rows = table.find_all('li')
    attributes = ['Class:', 'Socket', 'Clocks', 'Turbo ', 'Cores:', 'Typica']

    i = 0
    for row in rows:
        link = row.find('a').get('href')
        link = 'https://www.cpubenchmark.net/' + link

        soup = get_soup(link)
        price = row.find('span', class_='price-neww').get_text()
        name = row.find('span', class_='prdname').get_text()
        

        cpus[i]["name"] = name
        cpus[i]["price"] = price

        table = soup.find('div', class_= 'left-desc-cpu')
        body = table.find_all('p')
        for col in body:
            text = col.get_text()
            title = col.find('strong').get_text()

            #Verifying attribute is one that we want
            if title[:6] in attributes:
                data = text.split(':')

                #Edge case for cores/threads being in the same exact <p> tag
                if title == "Cores:":

                    #Another edge case for amd cpu's
                    if (text.find("(") != -1):
                        data[1] = data[1][1]
                        data.append("Threads")
                        data.append(data[1])

                    else:
                        temp2 = data[2]
                        data[1] = data[1][1]
                        data[2] = "Threads"
                        data.append(temp2[1:])

                    cpus[i][data[2]] = data[3]

                ##Formatting strings
                if title != "Socket:":
                    data[1] = data[1].replace(" ", "")
                else:
                    data[1] = data[1][1:]

                if (title=='Clockspeed:') or (title=='Turbo Speed:'):
                    data[1] = data[1][:-3]

                #Assign to dict
                cpus[i][data[0]] = data[1]
                
        #Gathering scores from other part of dataset
        table = soup.find('div', class_= 'right-desc')
        body = table.find_all('p')
        multi_threaded_score = table.find_all('span')[1].get_text()
        
        data = table.get_text().split(":")
        data[1] = data[1][:5]
        data[1] = data[1].replace(" ", "")
        single_threaded_score = data[1]
        print(name)
        
        cpus[i]["mt_score"] = int(multi_threaded_score)
        cpus[i]["st_score"] = int(single_threaded_score)

        if i >= n-1:
            break
        i = i+1
        time.sleep(sleep)
    
    return pd.DataFrame.from_dict(cpus)

Appending to ../src/datautil.py


Adding local src folder to sys.path

In [3]:
import sys
project_dir = '/home/atoris/course-project-thomas-wright/src'
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)


This next step actaully downloads all the data, this takes a while depnding on how fast you want to reload pages from the website, the rate limit is put into effect to not get blocked from the website.

In [4]:
import os
import pandas as pd
import datautil as du
import importlib

urls = ['https://www.cpubenchmark.net/high_end_cpus.html', 'https://www.cpubenchmark.net/mid_range_cpus.html', 'https://www.cpubenchmark.net/midlow_range_cpus.html', 'https://www.cpubenchmark.net/low_end_cpus.html']

df = pd.DataFrame()
for url in urls:
    df = pd.concat([df,du.download_data(url, 100, 1)])

df.reset_index(inplace=True)

AMD Ryzen Threadripper PRO 3995WX
AMD Ryzen Threadripper 3990X
AMD EPYC 7702
AMD EPYC 7702P
AMD EPYC 7742
AMD Ryzen Threadripper PRO 3975WX
AMD Ryzen Threadripper 3970X
AMD Ryzen Threadripper 3960X
AMD Ryzen 9 5950X
AMD EPYC 7502P
AMD EPYC 7452
AMD Ryzen Threadripper PRO 3955WX
AMD Ryzen 9 5900X
AMD Ryzen 9 3950X
AMD EPYC 7402P
Intel Xeon Gold 6248R @ 3.00GHz
Intel Xeon Platinum 8280 @ 2.70GHz
Intel Xeon W-3275M @ 2.50GHz
Intel Xeon Gold 6242R @ 3.10GHz
AMD Ryzen Threadripper PRO 3945WX
Intel Core i9-10980XE @ 3.00GHz
Intel Xeon Platinum 8168 @ 2.70GHz
AMD Ryzen 9 3900XT
AMD Ryzen 9 3900X
AMD EPYC 7302P
Intel Xeon W-2295 @ 3.00GHz
AMD Ryzen Threadripper 2990WX
Intel Core i9-9990XE @ 4.00GHz
Intel Core i9-9980XE @ 3.00GHz
Intel Xeon W-3175X @ 3.10GHz
AMD Ryzen 9 PRO 3900
Intel Xeon Gold 6248 @ 2.50GHz
Intel Core i9-9960X @ 3.10GHz
AMD Ryzen 9 3900
Intel Xeon Gold 6246R @ 3.40GHz
Intel Xeon W-3245 @ 3.20GHz
Intel Xeon Platinum 8259CL @ 2.50GHz
AMD Ryzen Threadripper 2950X
Intel Xeon W-32

Intel Core i7-2637M @ 1.70GHz
AMD Phenom II X3 B77
Intel Core i5-580M @ 2.67GHz
Intel Core2 Duo T8300 @ 2.40GHz
Athlon 64 Dual Core 5600+
Intel Pentium E5200 @ 2.50GHz
AMD Athlon II X2 4450e
Intel Core i3-2357M @ 1.30GHz
AMD A4-9120e
Intel Core2 Duo L9600 @ 2.13GHz
Intel Pentium P6000 @ 1.87GHz
Intel Celeron E3400 @ 2.60GHz
Intel Core2 Duo E6600 @ 2.40GHz
Intel Core i3-2365M @ 1.40GHz
Intel Core i3-2377M @ 1.50GHz
AMD Turion 64 X2 Mobile TL-68
AMD E2-3200 APU
AMD E2-9000e
Intel Xeon 5140 @ 2.33GHz
AMD Athlon 64 X2 Dual Core 5400+
AMD Athlon II X2 4300e
Athlon 64 Dual Core 5000+
Intel Pentium 3560Y @ 1.20GHz
AMD Turion II N530 Dual-Core
AMD A4-3310MX APU
Intel Pentium P6100 @ 2.00GHz
AMD TurionX2 Ultra DualCore Mobile ZM-85
AMD Opteron 275
Intel Pentium E2220 @ 2.40GHz
Intel Core2 Duo P8800 @ 2.66GHz
Intel Xeon E5205 @ 1.86GHz
Intel Xeon E5502 @ 1.87GHz
Intel Core i5-430UM @ 1.20GHz
Intel Core2 Duo P8600 @ 2.40GHz
Intel Pentium B960 @ 2.20GHz
Intel Pentium B940 @ 2.00GHz
AMD Athlon Dual

#### Verify the DataFrame

In [6]:
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   index        400 non-null    int64 
 1   name         400 non-null    object
 2   price        400 non-null    object
 3   Class        383 non-null    object
 4   Socket       382 non-null    object
 5   Clockspeed   400 non-null    object
 6   Turbo Speed  237 non-null    object
 7   Threads      400 non-null    object
 8   Cores        400 non-null    object
 9   Typical TDP  388 non-null    object
 10  mt_score     400 non-null    int64 
 11  st_score     400 non-null    int64 
dtypes: int64(3), object(9)
memory usage: 37.6+ KB


,index,name,price,Class,Socket,Clockspeed,Turbo Speed,Threads,Cores,Typical TDP,mt_score,st_score
0,0,AMD Ryzen Threadripper PRO 3995WX,NA,Desktop,sTRX4,2.7,4.2,128,6,280W,88673,2676
1,1,AMD Ryzen Threadripper 3990X,"$3,849.99",Desktop,sTRX4,2.9,4.3,128,6,280W,80508,2545
2,2,AMD EPYC 7702,"$6,229.00",Server,SP3,2.0,3.4,128,6,200W,71859,2096
3,3,AMD EPYC 7702P,"$4,088.00",Server,SP3,2.0,3.4,128,6,200W,68213,2210
4,4,AMD EPYC 7742,"$7,549.35",Server,SP3,2.3,3.4,128,6,225W,67185,2376


Functions for saving and reloading the dataframe

In [8]:
%%writefile -a ../src/datautil.py

def save_df(df, filename):
    basedir = os.path.dirname(os.getcwd())
    filepath = 'data/' + filename
    full_path = os.path.join(basedir, filepath)
    df.to_csv(full_path, index=False)

def load_data(url, file):
    file = os.path.dirname(os.getcwd()) + '/data/' + file
    print(file)
    if not os.path.exists(file):
        if not os.path.exists(os.path.dirname(file)):
            os.makedirs(os.path.dirname(file))
        df = download_data(url)
        save_df(df, file)
    return pd.read_csv(file)


Appending to ../src/datautil.py


Use these functions to save our data

In [9]:
importlib.reload(du)

du.save_df(df, 'cpu_data.csv')

Verify new project files have been saved

In [10]:
!tree ../

../
├── LICENSE
├── README.md
├── data
│   ├── cpu_data.csv
│   ├── cpu_data_cleaned.csv
│   └── cpu_data_encoded.csv
├── models
├── notebooks
│   ├── 0_setup_project_folders.ipynb
│   ├── 1_Retrieving_data.ipynb
│   ├── 2_Cleaning_data.ipynb
│   ├── 3_Simple_Graphs.ipynb
│   └── __pycache__
│       └── datautil.cpython-37.pyc
├── report
│   ├── interim.md
│   ├── proposal.md
│   └── report.md
└── src
    ├── __pycache__
    │   └── datautil.cpython-37.pyc
    └── datautil.py

7 directories, 15 files


Next step is to clean the rest of the data